In [2]:
# ───────────────────────────────────────────────────────────────────────────────
# 1. ENVIRONMENT SETUP
# ───────────────────────────────────────────────────────────────────────────────

# Install required packages (pinning versions to avoid 'super' object has no attribute 'run_server' error)
!pip install -q pandas numpy scikit-learn xgboost lightgbm shap plotly dash==2.11.1 kaggle dash-bootstrap-components werkzeug==2.2.3

# Import core libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    roc_auc_score, precision_score, recall_score,
    confusion_matrix, classification_report, roc_curve
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

# Explainability
import shap

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_white"

# Dashboard
# from jupyter_dash import JupyterDash # Removed this import
from dash import Dash, dcc, html, Input, Output # Use Dash directly
import dash_bootstrap_components as dbc

# Utilities
from datetime import datetime, timedelta
import json
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 4.7 MB/s eta 0:00:00


In [4]:
# ───────────────────────────────────────────────────────────────────────────────
# 2. Kaggle API Authentication & Dataset Download
# ───────────────────────────────────────────────────────────────────────────────

# Create kaggle directory
!mkdir -p ~/.kaggle

# Upload your kaggle.json file
from google.colab import files
# Uncomment the following lines to upload your kaggle.json file if not already done
print("📤 Please upload your kaggle.json file:")
uploaded = files.upload()
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download Telco Customer Churn dataset (classic SaaS-like churn dataset)
!kaggle datasets download -d blastchar/telco-customer-churn --unzip

📤 Please upload your kaggle.json file:


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/blastchar/telco-customer-churn
License(s): copyright-authors
  0% 0.00/172k [00:00<?, ?B/s]
100% 172k/172k [00:00<00:00, 546MB/s]


In [5]:
# ───────────────────────────────────────────────────────────────────────────────
# 3. DATA LOADING & UNDERSTANDING
# ───────────────────────────────────────────────────────────────────────────────

# Load the dataset
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

display(df.head())
display(df.describe())
display(df['Churn'].value_counts())

# Visualize churn distribution
fig = go.Figure()
churn_counts = df['Churn'].value_counts()
fig.add_trace(go.Bar(
    x=['Retained', 'Churned'],
    y=[churn_counts['No'], churn_counts['Yes']],
    marker_color=['#2ecc71', '#e74c3c'],
    text=[churn_counts['No'], churn_counts['Yes']],
    textposition='auto',
))
fig.update_layout(
    title='📊 Customer Churn Distribution',
    xaxis_title='Status',
    yaxis_title='Count',
    height=400
)
fig.show()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


,count
Churn,
No,5174
Yes,1869


In [6]:
# ───────────────────────────────────────────────────────────────────────────────
# 4. FEATURE ENGINEERING (USAGE & BEHAVIOR FOCUSED)
# ───────────────────────────────────────────────────────────────────────────────

# Create a copy for feature engineering
data = df.copy()

# Convert target to binary
data['Churn'] = (data['Churn'] == 'Yes').astype(int)

# Handle TotalCharges (convert to numeric)
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')
data['TotalCharges'].fillna(data['TotalCharges'].median(), inplace=True)

# ─── ENGINEERED FEATURES ───

# Average monthly spend (proxy for value derived)
data['AvgMonthlySpend'] = data['TotalCharges'] / (data['tenure'] + 1)

# Spending trajectory (are they spending more or less over time?)
data['SpendingTrend'] = data['MonthlyCharges'] / (data['AvgMonthlySpend'] + 1)
# > 1 = increasing, < 1 = decreasing (CHURN SIGNAL)

# Count of services used (more services = higher stickiness)
service_cols = ['PhoneService', 'MultipleLines', 'InternetService',
                'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport', 'StreamingTV', 'StreamingMovies']

# Create binary indicators
for col in service_cols:
    if col in data.columns:
        data[f'{col}_Used'] = (~data[col].isin(['No', 'No phone service', 'No internet service'])).astype(int)

# Total services adopted
data['TotalServicesUsed'] = data[[f'{col}_Used' for col in service_cols if f'{col}_Used' in data.columns]].sum(axis=1)

# Service density (services per month of tenure)
data['ServiceAdoptionRate'] = data['TotalServicesUsed'] / (data['tenure'] + 1)

# Is customer new? (higher churn in first 90 days)
data['IsNewCustomer'] = (data['tenure'] <= 3).astype(int)

# Customer lifecycle stage
data['LifecycleStage'] = pd.cut(data['tenure'],
                                 bins=[0, 6, 12, 24, 100],
                                 labels=['New', 'Growing', 'Mature', 'Champion'])

# Contract risk (month-to-month is high risk)
data['HighRiskContract'] = (data['Contract'] == 'Month-to-month').astype(int)

# Payment method risk (electronic check is often associated with churn)
data['RiskyPaymentMethod'] = (data['PaymentMethod'] == 'Electronic check').astype(int)

# Paperless billing adoption
data['PaperlessBilling_Binary'] = (data['PaperlessBilling'] == 'Yes').astype(int)

# Price sensitivity (high monthly charges relative to tenure)
data['PriceSensitivity'] = data['MonthlyCharges'] / (data['tenure'] + 1)

# Customer lifetime value (CLV)
data['CLV'] = data['TotalCharges']

# Value per service
data['ValuePerService'] = data['MonthlyCharges'] / (data['TotalServicesUsed'] + 1)


In [7]:
# ───────────────────────────────────────────────────────────────────────────────
# 5. MODELING PIPELINE
# ───────────────────────────────────────────────────────────────────────────────

# Prepare features for modeling

# Select features (exclude target and ID)
feature_cols = [col for col in data.columns if col not in
                ['customerID', 'Churn', 'LifecycleStage']]

# Handle categorical variables
categorical_cols = data[feature_cols].select_dtypes(include=['object']).columns.tolist()
numerical_cols = data[feature_cols].select_dtypes(include=['number']).columns.tolist()

# Encode categorical variables
data_encoded = data.copy()
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    data_encoded[col] = le.fit_transform(data_encoded[col].astype(str))
    label_encoders[col] = le

# Prepare X and y
X = data_encoded[feature_cols]
y = data_encoded['Churn']

# Time-aware split (simulate temporal data)
# Use tenure as proxy for time - train on older customers, validate on newer
train_idx = data['tenure'] > data['tenure'].quantile(0.2)
test_idx = ~train_idx

X_train = X[train_idx]
X_test = X[test_idx]
y_train = y[train_idx]
y_test = y[test_idx]

# ─── BASELINE MODEL ───
baseline = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
baseline.fit(X_train, y_train)
baseline_pred = baseline.predict_proba(X_test)[:, 1]
baseline_auc = roc_auc_score(y_test, baseline_pred)

# ─── ADVANCED MODEL (LightGBM) ───

# Handle class imbalance with scale_pos_weight
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

lgbm = lgb.LGBMClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=6,
    num_leaves=31,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    verbose=-1
)

lgbm.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='auc',
    callbacks=[lgb.early_stopping(50, verbose=False)]
)

# Predictions
y_pred_proba = lgbm.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba >= 0.5).astype(int)


In [8]:
# ───────────────────────────────────────────────────────────────────────────────
# 6. MODEL EVALUATION
# ───────────────────────────────────────────────────────────────────────────────

# Calculate metrics
auc = roc_auc_score(y_test, y_pred_proba)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'LightGBM (AUC={auc:.3f})',
                         line=dict(color='#3498db', width=3)))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random',
                         line=dict(color='gray', width=2, dash='dash')))
fig.update_layout(
    title='📈 ROC Curve - Model Performance',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    height=500,
    showlegend=True
)
fig.show()

# Confusion Matrix Heatmap
fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=['Predicted: Stay', 'Predicted: Churn'],
    y=['Actual: Stay', 'Actual: Churn'],
    colorscale='Blues',
    text=cm,
    texttemplate='%{text}',
    textfont={"size": 20},
))
fig.update_layout(
    title='🎯 Confusion Matrix',
    height=400
)
fig.show()


In [9]:
# ───────────────────────────────────────────────────────────────────────────────
# 7. EXPLAINABILITY (CRITICAL)
# ───────────────────────────────────────────────────────────────────────────────

# Initialize SHAP explainer
explainer = shap.TreeExplainer(lgbm)
shap_values = explainer.shap_values(X_test)

# Handle binary classification SHAP output
if isinstance(shap_values, list):
    shap_values_plot = shap_values[1]  # Churn class
else:
    shap_values_plot = shap_values

# Global Feature Importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': np.abs(shap_values_plot).mean(axis=0)
}).sort_values('importance', ascending=False).head(10)
display(feature_importance)

# SHAP Summary Plot
fig = go.Figure()
for i, (feat, imp) in enumerate(feature_importance.head(10).iterrows()):
    fig.add_trace(go.Bar(
        y=[feat],
        x=[imp['importance']],
        orientation='h',
        name=imp['feature'],
        text=f"{imp['importance']:.3f}",
        textposition='auto',
    ))

fig.update_layout(
    title='🎯 Top 10 Churn Drivers (SHAP Importance)',
    xaxis_title='Average Impact on Churn Prediction',
    yaxis_title='',
    height=500,
    showlegend=False,
    yaxis={'categoryorder': 'total ascending'}
)
fig.show()


,feature,importance
14,Contract,0.728595
38,ValuePerService,0.283331
36,PriceSensitivity,0.262581
31,ServiceAdoptionRate,0.117132
17,MonthlyCharges,0.110281
34,RiskyPaymentMethod,0.086841
20,SpendingTrend,0.061947
4,tenure,0.051586
33,HighRiskContract,0.049786
19,AvgMonthlySpend,0.048596


In [10]:
# ───────────────────────────────────────────────────────────────────────────────
# 8. INTERACTIVE DASHBOARD (CORE REQUIREMENT)
# ───────────────────────────────────────────────────────────────────────────────

# Prepare dashboard data
dashboard_data = data.iloc[X_test.index].copy()
dashboard_data['ChurnProbability'] = y_pred_proba
dashboard_data['RiskTier'] = pd.cut(
    y_pred_proba,
    bins=[0, 0.3, 0.7, 1.0],
    labels=['Low', 'Medium', 'High']
)

# Get SHAP values for all users
shap_df = pd.DataFrame(
    shap_values_plot,
    columns=X.columns,
    index=X_test.index
)

# Create Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP]) # Changed JupyterDash to Dash

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("🎯 SaaS Churn Intelligence Dashboard", className="text-center mb-4"),
            html.Hr()
        ])
    ]),

    # A. CHURN RISK OVERVIEW
    dbc.Row([
        dbc.Col([
            html.H3("📊 A. Churn Risk Overview"),
            dcc.Graph(id='risk-distribution')
        ], width=6),
        dbc.Col([
            html.H3("📈 Risk Tier Summary"),
            dcc.Graph(id='risk-tier-counts')
        ], width=6)
    ], className="mb-4"),

    # B. WHY USERS CHURN (GLOBAL)
    dbc.Row([
        dbc.Col([
            html.H3("🔍 B. Why Users Churn (Global Drivers)"),
            dcc.Dropdown(
                id='risk-filter',
                options=[
                    {'label': 'All Users', 'value': 'All'},
                    {'label': 'High Risk Only', 'value': 'High'},
                    {'label': 'Medium Risk Only', 'value': 'Medium'},
                    {'label': 'Low Risk Only', 'value': 'Low'}
                ],
                value='All',
                className="mb-3"
            ),
            dcc.Graph(id='global-drivers')
        ], width=12)
    ], className="mb-4"),

    # C. USER-LEVEL EXPLANATION
    dbc.Row([
        dbc.Col([
            html.H3("👤 C. User-Level Churn Explanation"),
            dcc.Dropdown(
                id='user-selector',
                options=[
                    {'label': f"Customer {cid} - {risk} Risk ({prob*100:.1f}%)",
                     'value': idx}
                    for idx, (cid, risk, prob) in enumerate(zip(
                        dashboard_data['customerID'],
                        dashboard_data['RiskTier'],
                        dashboard_data['ChurnProbability']
                    ))
                ],
                value=0,
                className="mb-3"
            ),
            html.Div(id='user-details')
        ], width=12)
    ], className="mb-4"),

    # D. BUSINESS INSIGHTS
    dbc.Row([
        dbc.Col([
            html.H3("💡 D. Business Insights & Recommendations"),
            html.Div(id='business-insights')
        ], width=12)
    ])
], fluid=True, style={'padding': '20px'})

# Callbacks
@app.callback(
    Output('risk-distribution', 'figure'),
    Input('risk-filter', 'value')
)
def update_risk_dist(risk_filter):
    filtered = dashboard_data if risk_filter == 'All' else dashboard_data[dashboard_data['RiskTier'] == risk_filter]

    fig = go.Figure()
    fig.add_trace(go.Histogram(
        x=filtered['ChurnProbability'],
        nbinsx=30,
        marker_color='#3498db',
        name='Churn Probability'
    ))
    fig.update_layout(
        title=f'Distribution of Churn Probabilities ({risk_filter} Risk)',
        xaxis_title='Churn Probability',
        yaxis_title='Count',
        height=400
    )
    return fig

@app.callback(
    Output('risk-tier-counts', 'figure'),
    Input('risk-filter', 'value')
)
def update_tier_counts(risk_filter):
    counts = dashboard_data['RiskTier'].value_counts()

    colors = {'Low': '#2ecc71', 'Medium': '#f39c12', 'High': '#e74c3c'}

    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=counts.index,
        y=counts.values,
        marker_color=[colors.get(tier, '#95a5a6') for tier in counts.index],
        text=counts.values,
        textposition='auto'
    ))
    fig.update_layout(
        title='Users by Risk Tier',
        xaxis_title='Risk Level',
        yaxis_title='Number of Users',
        height=400
    )
    return fig

@app.callback(
    Output('global-drivers', 'figure'),
    Input('risk-filter', 'value')
)
def update_global_drivers(risk_filter):
    if risk_filter == 'All':
        filtered_shap = shap_values_plot
    else:
        mask = dashboard_data['RiskTier'] == risk_filter
        filtered_shap = shap_values_plot[mask.values]

    importance = pd.DataFrame({
        'feature': X.columns,
        'importance': np.abs(filtered_shap).mean(axis=0)
    }).sort_values('importance', ascending=False).head(10)

    fig = go.Figure()
    fig.add_trace(go.Bar(
        y=importance['feature'],
        x=importance['importance'],
        orientation='h',
        marker_color='#e74c3c',
        text=importance['importance'].round(3),
        textposition='auto'
    ))
    fig.update_layout(
        title=f'Top 10 Churn Drivers ({risk_filter} Risk)',
        xaxis_title='Average SHAP Impact',
        yaxis={'categoryorder': 'total ascending'},
        height=500
    )
    return fig

@app.callback(
    Output('user-details', 'children'),
    Input('user-selector', 'value')
)
def update_user_details(user_idx):
    user = dashboard_data.iloc[user_idx]
    user_shap = shap_values_plot[user_idx]

    # Top 5 features
    top_idx = np.argsort(np.abs(user_shap))[-5:][::-1]
    top_features = [(X.columns[i], user_shap[i]) for i in top_idx]

    # Risk level styling
    prob = user['ChurnProbability']
    risk_color = '#e74c3c' if prob > 0.7 else '#f39c12' if prob > 0.3 else '#2ecc71'

    # Recommendation
    if prob > 0.7:
        recommendation = "🚨 URGENT: Schedule retention call + offer contract upgrade discount"
    elif prob > 0.3:
        recommendation = "⚠️ Monitor closely + send feature adoption email"
    else:
        recommendation = "✅ Maintain engagement with regular value communications"

    return dbc.Card([
        dbc.CardBody([
            html.H4(f"Customer: {user['customerID']}", className="mb-3"),
            html.H5([
                "Churn Probability: ",
                html.Span(f"{prob*100:.1f}%", style={'color': risk_color, 'font-weight': 'bold'})
            ]),
            html.H5([
                "Risk Tier: ",
                html.Span(f"{user['RiskTier']}", style={'color': risk_color, 'font-weight': 'bold'})
            ]),
            html.Hr(),
            html.H5("Top 5 Risk Factors:"),
            html.Ul([
                html.Li(f"{feat}: Impact = {shap_val:+.3f}")
                for feat, shap_val in top_features
            ]),
            html.Hr(),
            html.H5("📝 Recommended Action:"),
            html.P(recommendation, style={'font-size': '16px', 'font-weight': 'bold'})
        ])
    ])

In [11]:
@app.callback(
    Output('business-insights', 'children'),
    Input('risk-filter', 'value')
)
def update_insights(risk_filter):
    # These insights are simplified, the full logic for generating these would be complex
    # For demonstration, we'll just show some example insights
    insights = [
        f"🔹 {len(dashboard_data[dashboard_data['RiskTier']=='High']):,} users are at HIGH risk of churning",
        "🔹 Month-to-month contracts are a high churn indicator",
        "🔹 Low service adoption often leads to higher churn",
        "🔹 New customers (<6 months) have higher churn rates",
        "🔹 Declining spend trend is a strong predictor of churn",
        "💡 Recommended Strategy: Focus on contract upgrades for month-to-month users + improve onboarding"
    ]

    return dbc.Card([
        dbc.CardBody([
            html.H5("Key Findings:", className="mb-3"),
            html.Ul([html.Li(insight) for insight in insights])
        ])
    ], color="light")


In [12]:
# ───────────────────────────────────────────────────────────────────────────────
# 9. FINAL OUTPUT (PER USER)
# ───────────────────────────────────────────────────────────────────────────────

# Generate comprehensive user report
user_report = dashboard_data[['customerID']].copy()
user_report['ChurnProbability_%'] = (dashboard_data['ChurnProbability'] * 100).round(1)
user_report['RiskTier'] = dashboard_data['RiskTier']

# Get top 3 drivers per user
top_drivers_list = []
for idx in range(len(shap_values_plot)):
    user_shap = shap_values_plot[idx]
    top_3_idx = np.argsort(np.abs(user_shap))[-3:][::-1]
    drivers = [X.columns[i] for i in top_3_idx]
    top_drivers_list.append(', '.join(drivers))

user_report['TopDrivers'] = top_drivers_list

# Recommended actions
def get_recommendation(row):
    prob = row['ChurnProbability_%'] / 100
    if prob > 0.7:
        return "URGENT: Retention call + contract upgrade offer"
    elif prob > 0.5:
        return "Schedule check-in + product training"
    elif prob > 0.3:
        return "Send feature adoption campaign"
    else:
        return "Maintain engagement with value content"

user_report['RecommendedAction'] = user_report.apply(get_recommendation, axis=1)

display(user_report.head(10))

# Save to CSV
user_report.to_csv('churn_prediction_report.csv', index=False)


,customerID,ChurnProbability_%,RiskTier,TopDrivers,RecommendedAction
0,7590-VHVEG,52.2,Medium,"Contract, ValuePerService, PriceSensitivity",Schedule check-in + product training
2,3668-QPYBK,34.7,Medium,"Contract, ValuePerService, PriceSensitivity",Send feature adoption campaign
4,9237-HQITU,70.6,High,"Contract, ValuePerService, PriceSensitivity",URGENT: Retention call + contract upgrade offer
20,8779-QRDMV,62.7,Medium,"Contract, PriceSensitivity, ValuePerService",Schedule check-in + product training
22,1066-JKSGK,40.3,Medium,"Contract, ValuePerService, PriceSensitivity",Send feature adoption campaign
27,8665-UTDHZ,50.1,Medium,"Contract, ValuePerService, PriceSensitivity",Schedule check-in + product training
31,4929-XIHVW,70.6,High,"Contract, ValuePerService, PriceSensitivity",URGENT: Retention call + contract upgrade offer
33,7310-EGVHZ,40.3,Medium,"Contract, ValuePerService, PriceSensitivity",Send feature adoption campaign
34,3413-BMNZE,59.8,Medium,"Contract, PriceSensitivity, ValuePerService",Schedule check-in + product training
36,6047-YHPVI,64.3,Medium,"Contract, ValuePerService, PriceSensitivity",Schedule check-in + product training


In [13]:
# ───────────────────────────────────────────────────────────────────────────────
# 10. LAUNCH DASHBOARD
# ───────────────────────────────────────────────────────────────────────────────

# Execute the following command to launch the interactive dashboard:
app.run_server(mode='inline', height=1200)

<IPython.core.display.Javascript object>